In [1]:
!pip install pyconll

In [2]:
!pip install  evaluate  seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=b4784bba4a3614aca4d6aaa9995de2be93eec85ec90ccb89c47639446ca8c9ed
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
to

In [19]:
import pyconll
data = pyconll.load_from_file('/kaggle/input/arabic-pos/Arabic_POS.conllu')

review_sentences = []
pos_tags = []
unique_pos_tags = set()

for sentence in data:  
    sentence_tokens = []  
    sentence_pos = []
    
    for token in sentence:
        if token.upos is not None:  # Skip if POS is missing
            unique_pos_tags.add(token.upos)
            sentence_tokens.append(token.form)
            sentence_pos.append(token.upos)

    review_sentences.append(sentence_tokens)
    pos_tags.append(sentence_pos)


In [20]:

result = ' '.join(review_sentences[5])  
print(result)  


و تعذر ل متحدثة ب اسم وزارة الدفاع الالمانية ان تؤكد اليوم السبت هذه المعلومات .


In [21]:
labels=list(unique_pos_tags)


In [22]:
len(labels)

17

In [23]:
labels = ['None' if x is None else x for x in labels]


In [24]:
len(review_sentences[12]),len(pos_tags[12])

(12, 12)

In [25]:
import pandas as pd
df=pd.DataFrame({'tokens': review_sentences, 'pos_tags': pos_tags})
df.insert(0, 'id', range(1, len(df) + 1))

In [26]:
df.head()

,id,tokens,pos_tags
0,1,"[برلين, ترفض, حصول, شركة, اميركية, على, رخصة, ...","[X, VERB, NOUN, NOUN, ADJ, ADP, NOUN, NOUN, NO..."
1,2,"[برلين, 15, -, 7, (, اف, ب, ), -, افادت, صحيفة...","[X, NUM, PUNCT, NUM, PUNCT, X, X, PUNCT, PUNCT..."
2,3,"[و, في, نيسان, /, ابريل, الماضي, ،, تخلت, الدو...","[CCONJ, ADP, NOUN, PUNCT, NOUN, ADJ, PUNCT, VE..."
3,4,"[و, كانت, خسائر, المجموعة, الاسبانية, الرسمية,...","[CCONJ, VERB, NOUN, NOUN, ADJ, ADJ, VERB, ADP,..."
4,5,"[و, أشارت, صحيفة, الاحد, الى, ان, المستشار, شر...","[CCONJ, VERB, NOUN, NOUN, ADP, SCONJ, NOUN, X,..."


In [27]:
from sklearn.preprocessing import LabelEncoder

# Flatten the list of POS tags to fit encoder
all_pos_flat = [tag for sentence in pos_tags for tag in sentence]

le = LabelEncoder()
le.fit(all_pos_flat)

# Save mapping for later use (e.g., during inference)
id2label = {i: label for i, label in enumerate(le.classes_)}
label2id = {label: i for i, label in enumerate(le.classes_)}

# Encode the full dataset
df['pos_tags'] = [[label2id[tag] for tag in sentence] for sentence in pos_tags]


In [28]:
id2label

{0: 'ADJ',
 1: 'ADP',
 2: 'ADV',
 3: 'AUX',
 4: 'CCONJ',
 5: 'DET',
 6: 'INTJ',
 7: 'NOUN',
 8: 'NUM',
 9: 'PART',
 10: 'PRON',
 11: 'PROPN',
 12: 'PUNCT',
 13: 'SCONJ',
 14: 'SYM',
 15: 'VERB',
 16: 'X'}

In [29]:
df['tokens'][0]

['برلين',
 'ترفض',
 'حصول',
 'شركة',
 'اميركية',
 'على',
 'رخصة',
 'تصنيع',
 'دبابة',
 '"',
 'ليوبارد',
 '"',
 'الالمانية']

In [11]:
df.head()

,id,tokens,pos_tags
0,1,"[برلين, ترفض, حصول, شركة, اميركية, على, رخصة, ...","[16, 15, 7, 7, 0, 1, 7, 7, 7, 12, 16, 12, 0]"
1,2,"[برلين, 15, -, 7, (, اف, ب, ), -, افادت, صحيفة...","[16, 8, 12, 8, 12, 16, 16, 12, 12, 15, 7, 7, 0..."
2,3,"[و, في, نيسان, /, ابريل, الماضي, ،, تخلت, الدو...","[4, 1, 7, 12, 7, 0, 12, 15, 7, 0, 1, 7, 12, 16..."
3,4,"[و, كانت, خسائر, المجموعة, الاسبانية, الرسمية,...","[4, 15, 7, 7, 0, 0, 15, 1, 7, 8, 0, 4, 15, 8, ..."
4,5,"[و, أشارت, صحيفة, الاحد, الى, ان, المستشار, شر...","[4, 15, 7, 7, 1, 13, 7, 16, 15, 13, 7, 7, 0, 1..."


In [12]:
len(df['tokens'][0])

13

In [13]:
df['pos_tags'][0]

[16, 15, 7, 7, 0, 1, 7, 7, 7, 12, 16, 12, 0]

In [30]:
from datasets import Dataset
from sklearn.model_selection import train_test_split


train_df, tests = train_test_split(df, test_size=0.3, random_state=42)
val_df,test_df=  train_test_split(tests, test_size=0.2, random_state=42)

In [31]:
train_dataset = Dataset.from_pandas(train_df,preserve_index=False)
test_dataset = Dataset.from_pandas(test_df, preserve_index=False)
val_dataset=Dataset.from_pandas(val_df, preserve_index=False)

In [32]:
train_dataset

Dataset({
    features: ['id', 'tokens', 'pos_tags'],
    num_rows: 4252
})

In [17]:

#id2label = {i: label for i, label in enumerate(labels)}
#label2id = {label: i for i, label in enumerate(labels)}


In [34]:
from transformers import AutoTokenizer, TFAutoModelForTokenClassification

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")

# Load model with proper label mapping
model = TFAutoModelForTokenClassification.from_pretrained(
    "asafaya/bert-base-arabic",
    num_labels=len(le.classes_),        
    id2label=id2label,             
    label2id=label2id              
)


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/334k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

2025-05-09 12:36:35.075466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746794195.301451      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746794195.363428      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

I0000 00:00:1746794211.326096      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"pos_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [36]:
tokenized_train = train_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/4252 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [37]:
# Check the structure of the tokenized dataset
print("Tokenized train dataset features:", tokenized_train.column_names)

# Access a sample from the tokenized dataset to see the tokenized inputs and labels
sample = tokenized_train[0]
print("\nSample tokenized input:")
print("Tokens:", sample['input_ids'])  # Tokenized IDs
print("Attention Mask:", sample['attention_mask'])  # Attention Mask (if used)
print("Labels:", sample['labels'])  # Aligned POS tags

# Optionally, convert the token IDs back to words using the tokenizer for readability
print("\nDecoded tokens:")
decoded_tokens = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
print(decoded_tokens)


Tokenized train dataset features: ['id', 'tokens', 'pos_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels']

Sample tokenized input:
Tokens: [2, 4539, 4445, 30435, 6734, 2061, 2430, 3]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1]
Labels: [-100, 7, 7, 7, -100, 1, 7, -100]

Decoded tokens:
ارتفاع نسبة المدخنات فى السعودية


In [38]:
tokenized_test = test_dataset.map(tokenize_and_align_labels, batched=True)



Map:   0%|          | 0/365 [00:00<?, ? examples/s]

In [39]:
tokenized_val = val_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1458 [00:00<?, ? examples/s]

In [40]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

In [41]:
import evaluate

seqeval = evaluate.load("seqeval")

In [42]:
example = train_dataset[0]

In [44]:
label_list=le.classes_

In [45]:
import numpy as np

labels = [label_list[i] for i in example[f"pos_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [46]:
from transformers import create_optimizer

batch_size = 16
num_train_epochs = 3
num_train_steps = (len(train_dataset) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [47]:
print(model)

In [48]:
tf_train_set = model.prepare_tf_dataset (
   tokenized_train,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset (
    tokenized_val,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)
tf_test_set = model.prepare_tf_dataset (
    tokenized_test,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)


In [49]:
import tensorflow as tf

model.compile(optimizer=optimizer)  

In [50]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [51]:
!git config --global user.email "lolo.hussien861@gmail.com"
!git config --global user.name "Alaa hussien"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [52]:
from huggingface_hub import notebook_login

notebook_login()

In [53]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="Part_of_speech_arabic",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/AlaaHussien/Part_of_speech_arabic into local empty directory.


In [54]:
callbacks = [metric_callback,push_to_hub_callback]

In [55]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

Epoch 1/3
265/265 [==============================] - ETA: 0s - loss: 0.4216

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PUNCT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: 

265/265 [==============================] - 140s 408ms/step - loss: 0.4216 - val_loss: 0.1348 - precision: 0.9514 - recall: 0.9449 - f1: 0.9482 - accuracy: 0.9634
Epoch 2/3
265/265 [==============================] - ETA: 0s - loss: 0.1180

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PUNCT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: 

265/265 [==============================] - 106s 399ms/step - loss: 0.1180 - val_loss: 0.1149 - precision: 0.9579 - recall: 0.9522 - f1: 0.9551 - accuracy: 0.9686
Epoch 3/3
265/265 [==============================] - ETA: 0s - loss: 0.0974

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PUNCT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: 

265/265 [==============================] - 105s 398ms/step - loss: 0.0974 - val_loss: 0.1114 - precision: 0.9599 - recall: 0.9546 - f1: 0.9572 - accuracy: 0.9701


In [56]:
loss=model.evaluate(tf_test_set)
print(loss)

23/23 [==============================] - 2s 96ms/step - loss: 0.1277
0.12770117819309235


In [75]:
result = ' '.join(review_sentences[0])  
print(result)  

برلين ترفض حصول شركة اميركية على رخصة تصنيع دبابة " ليوبارد " الالمانية


In [73]:
result = ' '.join(pos_tags[0])  
print(result)

X VERB NOUN NOUN ADJ ADP NOUN NOUN NOUN PUNCT X PUNCT ADJ


In [76]:
from transformers import pipeline

pipe = pipeline("token-classification", model="AlaaHussien/Part_of_speech_arabic")
pipe(result)

Some layers from the model checkpoint at AlaaHussien/Part_of_speech_arabic were not used when initializing TFBertForTokenClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at AlaaHussien/Part_of_speech_arabic.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.
Device set to use 0
Asking to truncate to max_length but n

[{'entity': 'X',
  'score': 0.9930369,
  'index': 1,
  'word': 'برلين',
  'start': 0,
  'end': 5},
 {'entity': 'VERB',
  'score': 0.99469537,
  'index': 2,
  'word': 'ترفض',
  'start': 6,
  'end': 10},
 {'entity': 'NOUN',
  'score': 0.9884835,
  'index': 3,
  'word': 'حصول',
  'start': 11,
  'end': 15},
 {'entity': 'NOUN',
  'score': 0.9978123,
  'index': 4,
  'word': 'شركة',
  'start': 16,
  'end': 20},
 {'entity': 'ADJ',
  'score': 0.9831988,
  'index': 5,
  'word': 'اميركية',
  'start': 21,
  'end': 28},
 {'entity': 'ADP',
  'score': 0.99734026,
  'index': 6,
  'word': 'على',
  'start': 29,
  'end': 32},
 {'entity': 'NOUN',
  'score': 0.99659014,
  'index': 7,
  'word': 'رخصة',
  'start': 33,
  'end': 37},
 {'entity': 'NOUN',
  'score': 0.9923678,
  'index': 8,
  'word': 'تصنيع',
  'start': 38,
  'end': 43},
 {'entity': 'NOUN',
  'score': 0.9768653,
  'index': 9,
  'word': 'دب',
  'start': 44,
  'end': 46},
 {'entity': 'NOUN',
  'score': 0.9731041,
  'index': 10,
  'word': '##ابة',


In [67]:
text="محمد ذهب الى الملعب"

In [68]:
from transformers import pipeline

pipe = pipeline("token-classification", model="AlaaHussien/Part_of_speech_arabic")
pipe(text)

Some layers from the model checkpoint at AlaaHussien/Part_of_speech_arabic were not used when initializing TFBertForTokenClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at AlaaHussien/Part_of_speech_arabic.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.
Device set to use 0
Asking to truncate to max_length but n

[{'entity': 'X',
  'score': 0.86801434,
  'index': 1,
  'word': 'محمد',
  'start': 0,
  'end': 4},
 {'entity': 'VERB',
  'score': 0.98758763,
  'index': 2,
  'word': 'ذهب',
  'start': 5,
  'end': 8},
 {'entity': 'ADP',
  'score': 0.9981111,
  'index': 3,
  'word': 'الى',
  'start': 9,
  'end': 12},
 {'entity': 'NOUN',
  'score': 0.9909791,
  'index': 4,
  'word': 'الملعب',
  'start': 13,
  'end': 19}]